In [1]:
import numpy as np 
import pandas as pd 
import os 
from tqdm import tqdm 
from sklearn import metrics
import cv2 

In [2]:
df = pd.read_csv('./xgb_pre/model/xgb_G_oof.csv')
df['vid'] = df['contact_id'].apply(lambda x: '_'.join(x.split('_')[:2]))
df['step'] = df['contact_id'].apply(lambda x: int(x.split('_')[2]))
df = df[df['pred']>0.001]
print(df.shape)

det_dict = np.load('./data/det_dict.npy', allow_pickle=True).item()

(261061, 7)


In [3]:
df.head(3)

,contact_id,fold,contact,pred,frame,vid,step
0,58188_001358_0_46075_G,2,0,0.001174,291.368,58188_001358,0
3,58188_001358_0_42477_G,2,0,0.001358,291.368,58188_001358,0
4,58188_001358_0_47973_G,2,0,0.001093,291.368,58188_001358,0


In [4]:
def get_sample(vid, window_size = 20, stride=4, out_size = 128,):
    ws = []
    hs = []
    
    # fr_id is row['frame']
        # fr_id => 291 
        # window_size => [-54, -48, -42, -36, -30, -24, -18, -13, -8, -4, -2, 0, 2, 4, 8, 13, 18, 24, 30, 36, 42, 48, 54]

    for fr in range(fr_id + window_size[0], fr_id + window_size[-1] + 1):
        # for in range(237, 346)
        if fr in det_dict[vid]:
            # idx1 is nfl_player_id_1.
            if idx1 in det_dict[vid][fr]: 
                # co-ordinate and width and height of helmet.
                x, y, w, h = det_dict[vid][fr][idx1]['box']
                ws.append(w)
                hs.append(h)

    if len(ws)>0:
        # for particular vid and player, image crop_size = 4 times the average helmet size.
        crop_size = int(4*max(np.mean(ws), np.mean(hs)))
            # crop_size => 83
    else:
        crop_size = out_size
    
    
    bboxes = []
    for fr in range(fr_id + window_size[0], fr_id + window_size[-1] + 1):
        if fr in det_dict[vid]:
            if idx1 in det_dict[vid][fr]: 
                x, y, w, h = det_dict[vid][fr][idx1]['box']
                # xc, yc => center co-ordinate of helmet.
                xc = x + w/2
                yc = y + h/2

                bboxes.append([xc-crop_size, yc-crop_size, xc+crop_size, yc+crop_size])
            else:
                bboxes.append([np.nan, np.nan, np.nan, np.nan])
        else:
            bboxes.append([np.nan, np.nan, np.nan, np.nan])


        # pd.DataFrame(bboxes).head(3) =>
        #     0   1   2   3
        # 0 NaN NaN NaN NaN
        # 1 NaN NaN NaN NaN
        # 2 NaN NaN NaN NaN
    bboxes = pd.DataFrame(bboxes).interpolate(limit_direction='both').values
    # interpolate(method='linear') => Fill NaN values using an interpolation method.
    # ‘linear’ => Ignore the index and treat the values as equally spaced. 
    # axis = 0 => Axis to interpolate along.
    # limit_direction='both' => Consecutive NaNs will be filled in this direction.
        # bboxes =>
        # [[806.5 260.  972.5 426. ]
        #  [806.5 260.  972.5 426. ]
        #  [806.5 260.  972.5 426. ]


    images = []
    masks1 = []
    empty_count = 0
    
        # bboxes.sum() => 262589.0
    if bboxes.sum() > 0: 
        for i, ii in enumerate(window_size):
            # i, ii => 0, -54        
            fr = ii + fr_id
                # fr (in loop runs)
                # 237, 243, 249, 255, 261, 267, 273, 278, 283, 287, 289, 291, 293, 295, 299, 304, 309, 315, 321, 327, 333, 339, 345            
            path = f'{vid}_{fr}'
            
            if path in image_dict:
                image = image_dict[path]
                    # image.shape => (720,1280,3)
            else:
                image = np.ones((720, 1280,3), dtype = np.uint8)
                empty_count +=1

            mask1 = np.ones((720, 1280), dtype = np.uint8)

            # x1, y1, x2, y2 = list(map(int, bboxes[i]))
            # map() function executes a specified function for each item in an iterable. The item is sent to the function as a parameter. 
            x1, y1, x2, y2 = list(map(int, bboxes[ii-window_size[0]]))
                # ii-window_size[0] => 0
                # bboxes[ii-window_size[0]] => [806.5 260.  972.5 426. ]
                # list(map(int, bboxes[ii-window_size[0]])) => [806, 260, 972, 426]             
            
            # random change as previous x2 and y2 are not satisfactory.
            y1 = y1 + int(0.2*crop_size)
                # crop_size => 83
                # int(0.2*crop_size) => 16
            x2 = x1 + crop_size*2
            y2 = y1 + crop_size*2

            if fr in det_dict[vid]:
                if idx1 in det_dict[vid][fr]: 
                    x, y, w, h = det_dict[vid][fr][idx1]['box']
                    # cv2.rectangle(image, (x, y), (x+w, y+h), (0,0,255), 2)
                    # mask1[y:y+h, x:x+w] = 255
                    _ = cv2.circle(mask1, (x+w//2, y+h//2), int(0.3*h+0.3*w), 255, thickness=-1) 
                        # draw circle over mask (inplace).

            
            # y first as image is in H,W format.
            crop = image[y1:y2, x1:x2]
            crop_mask1 = mask1[y1:y2, x1:x2]
                # crop.shape => (166, 166, 3)
                # crop.shape[:2] => (166, 166)
            cr_y, cr_x = crop.shape[:2]
            
            if cr_x == crop_size*2 and cr_y == crop_size*2:
                    # crop_size => 83
                crop = cv2.resize(crop, (out_size*2,out_size*2))
                    # crop.shape => (256, 256, 3)
                crop_mask1 = cv2.resize(crop_mask1, (out_size*2,out_size*2))
                images.append(crop)
                masks1.append(crop_mask1)
            else:                
                    # idx1, vid, tmp_crop.shape, fr_id => 46171, 58188_001358_Endzone, (226, 226, 3), 315                
                tmp_crop =  np.ones((crop_size*2, crop_size*2,3), dtype = np.uint8)
                    # tmp_crop.shape => (226, 226, 3)
                tmp_mask1 =  np.ones((crop_size*2, crop_size*2), dtype = np.uint8)
                    # tmp_mask1.shape => (226, 226)
                    
                if x1 < 0:
                    if y2>=720:
                        # x1,x2,y1,y2 => -61, 131, 530, 722
                        # cr_y, crop_size, cr_x => 190, 96, 0
                        # crop_size*2-cr_y => 2
                        # crop_size*2-cr_x => 192
                        # crop.shape => (190, 0, 3)
                        # tmp_crop.shape => (192, 192, 3) 
                        
                        tmp_crop[crop_size*2-cr_y:,:cr_x] = crop
                        tmp_mask1[crop_size*2-cr_y:,:cr_x] = crop_mask1
                    else:
                        # x1,x2,y1,y2 => -5, 193, 194, 392
                        # cr_y, crop_size, cr_x => 198, 99, 0
                        # crop_size*2-cr_y => 0
                        # crop_size*2-cr_x => 198
                        # crop.shape => (198, 0, 3)
                        # tmp_crop.shape => (198, 198, 3)
                  
                        tmp_crop[:cr_y,:cr_x] = crop
                        tmp_mask1[:cr_y,:cr_x] = crop_mask1

                elif x2> 1280:
                    if y2>=720:
                        # x1,x2,y1,y2 => 1167, 1287, 602, 722
                        # cr_y, crop_size, cr_x => 118, 60, 113
                        # crop_size*2-cr_y => 2
                        # crop_size*2-cr_x => 7
                        # crop.shape => (118, 113, 3)
                        # tmp_crop.shape => (120, 120, 3)
                        
                        tmp_crop[crop_size*2-cr_y:,crop_size*2-cr_x:] = crop
                        tmp_mask1[crop_size*2-cr_y:,crop_size*2-cr_x:] = crop_mask1                        
                    else:
                        # x1,x2,y1,y2 => 1058, 1284, 199, 425
                        # cr_y, crop_size, cr_x => 226, 113, 222
                        # crop_size*2-cr_x  => 4
                        # crop.shape => (226, 222, 3)
                        # tmp_crop.shape => (226, 226, 3) 
                        
                        tmp_crop[:cr_y,crop_size*2-cr_x:] = crop                        
                        tmp_mask1[:cr_y,crop_size*2-cr_x:] = crop_mask1

                tmp_crop = cv2.resize(tmp_crop, (out_size*2,out_size*2))
                    # tmp_crop.shape => (256, 256, 3)
                tmp_mask1 = cv2.resize(tmp_mask1, (out_size*2,out_size*2))
                    # tmp_mask1.shape => (256, 256)
                images.append(tmp_crop)
                masks1.append(tmp_mask1)
    else:
        for i, ii in enumerate(window_size):        
            empty_count +=1
            crop =  np.ones((out_size*2, out_size*2,3), dtype = np.uint8)
                # crop.shape => (256, 256, 3)
            crop_mask1 =  np.ones((out_size*2, out_size*2), dtype = np.uint8)
                # crop_mask1.shape => (256, 256)
            images.append(crop)
            masks1.append(crop_mask1)
            
    return images, masks1, empty_count


In [5]:
# idx1=46171
# fr_id=315
# _ = get_sample('58188_001358_Endzone', window_size = [-54, -48, -42, -36, -30, -24, -18, -13, -8, -4, -2, 0, 2, 4, 8, 13, 18, 24, 30, 36, 42, 48, 54], stride=4)

### Checking which videos contains which fold

In [6]:
## data is heavily imbalanced so taking minority value.
# T1 = df[df['contact']==1]
# vids = df[df['contact']==1]['vid'].unique()
# print(len(vids))

In [7]:
## 0-32 fold2
## 32:70 fold1
## 70:98 fold0
# vids_ = vids[0:32]
#T1[T1['vid'].isin(vids_)]['fold'].unique()

### Slicing

In [9]:
os.makedirs('slicing_g/', exist_ok=True)

In [10]:
vids = df[df['contact']==1]['vid'].unique()
## 0-32 fold2
## 32:70 fold1
## 70:98 fold0
vids = np.concatenate([np.random.choice(vids[0:32], replace=False, size=25),
                np.random.choice(vids[32:70], replace=False, size=25),
                np.random.choice(vids[70:98], replace=False, size=25)])
print(len(vids))


df = df[df['vid'].isin(vids)]
print(len(df))

75


In [11]:
results = []

In [12]:
window_size = [-54, -48, -42, -36, -30, -24, -18, -13, -8, -4, -2, 0, 2, 4, 8, 13, 18, 24, 30, 36, 42, 48, 54]

In [13]:

for vid in tqdm(vids):

    e_vid = vid + f'_Endzone'
    s_vid = vid + f'_Sideline'

    e_vid_path = f'./data/train/{e_vid}.mp4'
    s_vid_path = f'./data/train/{s_vid}.mp4'
    
    image_dict = {}
    
    cap = cv2.VideoCapture(e_vid_path)
        # cap.get(cv2.CAP_PROP_FPS) => 59.94005994005994 (frames per second)
        # cap.get(cv2.CAP_PROP_POS_FRAMES) => 0.0

    # set video property cv2.CAP_PROP_POS_FRAMES => set the current frame position to 230
    # POS => point of start
    cap.set(cv2.CAP_PROP_POS_FRAMES,230)
    frame_count = 230
    
    while 1:
        # cap.read() => grabs, decodes and returns the next video frame.
        # ret => false if no frames has been grabbed
        ret, frame = cap.read()
            # ret => True
            # type(frame), frame.shape => <class 'numpy.ndarray'> (720, 1280, 3)            

        if not ret: break# exit the loop
        kk = f'{e_vid}_{frame_count}'
            # kk => 58188_001358_Endzone_230
        image_dict[kk] = frame
        frame_count +=1

    # frame_count => 783          
    cap = cv2.VideoCapture(s_vid_path)
    cap.set(cv2.CAP_PROP_POS_FRAMES, 230)
    frame_count = 230
    while 1:
        ret, frame = cap.read()
        if not ret: break
        kk = f'{s_vid}_{frame_count}'
        image_dict[kk] = frame
        frame_count +=1

    # frame_count => 783  
    df1 = df[df['vid']==vid].reset_index(drop=True)
        # df1.head() =>
        #                contact_id  fold  contact      pred    frame           vid  step  
        # 0  58188_001358_0_42477_G     2        0  0.001075  291.368  58188_001358     0
        # 1  58188_001358_0_52907_G     2        0  0.001729  291.368  58188_001358     0 
        # 2  58188_001358_0_46106_G     2        0  0.004515  291.368  58188_001358     0         
    
    
    for i, row in df1.iterrows():
    # for i, row in tqdm(df1.iterrows()):
        idx2 = 'G'

        idx1 = int(row['contact_id'].split('_')[-2])
            # idx1 42477

        fr_id = int(row['frame'])
        step = int(row['step'])
        
            # e_vid => '58188_001358_Endzone'

        
        e_images, e_masks1, e_empty_count = get_sample(e_vid, window_size = window_size, stride=4, out_size = 128)
            # type(e_images) => <class 'list'>
            # len(e_images) => 23 
        s_images, s_masks1, s_empty_count = get_sample(s_vid, window_size = window_size, stride=4, out_size = 128)
        

        # ### visualize
        # ## ii = 22
        # os.makedirs(f'draw/slicing_g/{idx1}_{idx2}_{fr_id}', exist_ok=True)
        # for ii in range(len(e_images)):
        #     e_img = e_images[ii]

        #     # setting circle_mask values to 255
        #     e_img[e_masks1[ii]>100] = 255

        #     s_img = s_images[ii]

        #     # setting circle_mask values to 255
        #     s_img[s_masks1[ii]>100] = 255

        #     ## e_img = cv2.resize(e_img, (256,256))
        #     ## s_img = cv2.resize(s_img, (256,256))
        #     e_img = np.hstack([e_img, s_img])
        #     cv2.imwrite(f'draw/slicing_g/{idx1}_{idx2}_{fr_id}/{ii}.jpg', e_img)

            # fr_id => 291
            # f'{fr_id:04d}' => 0291
            
        path = f'slicing_g/{vid}_{idx1}_{idx2}_{fr_id:04d}_{step}' 
            # 'distance': row['pred']
        item = {'path': path, 'fold':row['fold'], 'contact': row['contact'], 'distance': row['pred'], 'step': step, 'e_empty': e_empty_count, 's_empty': s_empty_count}
        results.append(item)

        images_e = []
        images_s = []
        for ii in range(len(e_images)):
            e_img = e_images[ii]
            
            # setting circle_mask values to 255
            e_img[e_masks1[ii]>100] = 255

            s_img = s_images[ii]
            
            # setting circle_mask values to 255
            s_img[s_masks1[ii]>100] = 255

            images_e.append(e_img)
            images_s.append(s_img)
            

        np.save(f'{path}_e.npy', np.array(images_e))
        np.save(f'{path}_s.npy', np.array(images_s))

100%|████████████████████████████████████████| 75/75 [3:23:46<00:00, 163.02s/it]


In [14]:
df = pd.DataFrame(results)
print(df.shape)
print(df.head(3))

df.to_csv('slicing_g.csv', index=False)

(78204, 7)
                                    path  fold  contact  distance  step  \
0  slicing_g/58291_001415_41500_G_0295_0     2        0  0.012337     0   
1  slicing_g/58291_001415_47872_G_0295_0     2        0  0.004777     0   
2  slicing_g/58291_001415_37145_G_0295_0     2        0  0.002074     0   

   e_empty  s_empty  
0       23        0  
1        0        0  
2        0        0  


In [15]:
df['fold'].unique()

array([2, 1, 0])

In [16]:
# # plotter 
# plt.rcParams["figure.figsize"] = (20,8)
# fx, arr = plt.subplots(1,1) 
# arr.imshow(img)   